In [ ]:
# %% Deep learning - Section 12.125
#    Save and load trained model

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [ ]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import time

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Data

# Load data
data = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'),delimiter=',')

# Split labels from data
labels = data[:,0]
data   = data[:,1:]

# Normalise data (original range is (0,255))
data_norm = data / np.max(data)


In [ ]:
# %% Create train and test datasets

# Convert to tensor (float and integers)
data_tensor   = torch.tensor(data_norm).float()
labels_tensor = torch.tensor(labels).long()

# Split data with scikitlearn (10% test data)
train_data,test_data,train_labels,test_labels = train_test_split(data_tensor,labels_tensor,test_size=0.1)

# Convert to PyTorch datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Convert into DataLoader objects
batch_size   = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])


In [ ]:
# %% Model class

# %% Function to generate the model
#    Keep flexibly loop over model depth/breadth, add flexibility over optimizer

def gen_model():

    class mnist_FFN(nn.Module):
        def __init__(self):
            super().__init__()

            # Architecture
            self.input  = nn.Linear(784,64)
            self.hid1   = nn.Linear( 64,32)
            self.hid2   = nn.Linear( 32,32)
            self.output = nn.Linear( 32,10)

        # Forward propagation
        def forward(self,x):

            x = F.relu(self.input(x))
            x = F.relu(self.hid1(x))
            x = F.relu(self.hid2(x))
            x = torch.log_softmax(self.output(x),axis=1)

            return x

    # Create model instance
    ANN = mnist_FFN()

    # Loss function
    loss_fun = nn.CrossEntropyLoss()

    # Optimizer
    optimizer = torch.optim.SGD(ANN.parameters(),lr=0.01)

    return ANN,loss_fun,optimizer


In [ ]:
# %% Function to train the model

def train_model():

    # Parameters, model instance, inizialise vars
    num_epochs = 10
    ANN,loss_fun,optimizer = gen_model()

    losses    = []
    train_acc = []
    test_acc  = []

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Loop over training batches
        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Forward propagation and loss
            yHat = ANN(X)
            loss = loss_fun(yHat,y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Loss and accuracy from this batch
            batch_loss.append( loss.item() )
            batch_acc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

        losses.append( np.mean(batch_loss) )
        train_acc.append( np.mean(batch_acc) )

        # Test accuracy
        ANN.eval()
        with torch.no_grad():
            X,y = next(iter(test_loader))
            yHat = ANN(X)
            test_acc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )
        ANN.train()

    return train_acc,test_acc,losses,ANN


In [ ]:
# %% Run the model

train_acc,test_acc,losses,ANN = train_model()


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5)) / 2
fig,ax = plt.subplots(1,2,figsize=(1.5*phi*6,6))

ax[0].plot(losses)
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].set_ylim([0,3])
ax[0].set_title('Model loss')

ax[1].plot(train_acc,label='Train')
ax[1].plot(test_acc,label='Test')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy (%)')
ax[1].set_ylim([10,100])
ax[1].set_title(f'Final model test accuracy: {test_acc[-1]:.2f}%')
ax[1].legend()

plt.savefig('figure35_save_load_trained_models.png')

plt.show()

files.download('figure35_save_load_trained_models.png')


In [ ]:
# %% Save trained model

torch.save(ANN.state_dict(),'trained_model.pt')
files.download('trained_model.pt')


In [ ]:
# %% Load the model under a different name

# Generate 2 instances of the same model
model_1 = gen_model()[0]
model_2 = gen_model()[0]

# Replace one model's weights with those of the trained model
model_1.load_state_dict(torch.load('trained_model.pt'))


In [ ]:
# %% Show that they are actually the same

# Get test data
X,y = next(iter(test_loader))

# Predict with the base model and with the 2 new models, same data
yhat_0 = ANN(X)
yHat_1 = model_1(X)
yHat_2 = model_2(X)

# Plotting (output node number 6)
phi = (1 + np.sqrt(5)) / 2
fig = plt.figure(figsize=(1.5*phi*6,6))

plt.plot(yhat_0[:,5].detach(),'b',label='Original model')
plt.plot(yHat_1[:,5].detach(),'ro',label='Reloaded model')
plt.plot(yHat_2[:,5].detach(),'mx',label='Not loaded model')

plt.legend()
plt.xlabel('Stimulus index')
plt.ylabel('Model output for node "6"')
plt.xlim([1000,1100])

plt.savefig('figure36_save_load_trained_models.png')

plt.show()

files.download('figure36_save_load_trained_models.png')
